# Advanced Querying Mongo

Importing libraries and setting up connection

In [12]:
from pymongo import MongoClient
from pymongo import ASCENDING, DESCENDING

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [7]:
# Your Code
cursor = db.companies.find({"name":"Babelgum"},{"name":1,"_id":0})
for name in cursor:
    print(name)
    break  # para mostrar solo el primero

{'name': 'Babelgum'}


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [13]:
# Your Code
cursor = db.companies.find({"number_of_employees": {"$gt":5000}}).sort([("number_of_employees", ASCENDING)]).limit(20)
for company in cursor:
    print(company)
    break # para mostrar solo el primero

{'_id': ObjectId('52cdef7c4bab8bd675297e49'), 'name': 'Nintendo', 'permalink': 'nintendo', 'crunchbase_url': 'http://www.crunchbase.com/company/nintendo', 'homepage_url': 'http://nintendo.com', 'blog_url': '', 'blog_feed_url': '', 'twitter_username': 'NintendoAmerica', 'category_code': 'games_video', 'number_of_employees': 5080, 'founded_year': 1889, 'founded_month': 9, 'founded_day': 23, 'deadpooled_year': None, 'deadpooled_month': None, 'deadpooled_day': None, 'deadpooled_url': None, 'tag_list': 'games, console', 'alias_list': '', 'email_address': 'noalegal@noa.nintendo.com', 'phone_number': '1-800-255-3700 ', 'description': '', 'created_at': 'Sat Dec 22 13:27:00 UTC 2007', 'updated_at': 'Thu Sep 05 12:41:16 UTC 2013', 'overview': '<p>Nintendo, a technology company widely known for its line of game consoles, was actually founded in 1889 by Fusajiro Yamauchi. It began as a card game company and evolved into one of the largest Japanese companies with a Market Capitalization of over $85

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [18]:
# Your Code
def companiesYearF(year):
    query = db.companies.find({"founded_year": year},{"name":1,"founded_year":1,"_id":0})
    return query
    
for i in range(2000,2006):
    cursor = companiesYearF(i)
    for company in cursor:
        print(company)
        break
# Solo un break para que imprima el primero de cada año

{'name': 'AllofMP3', 'founded_year': 2000}
{'name': 'TechnologyGuide', 'founded_year': 2001}
{'name': 'StumbleUpon', 'founded_year': 2002}
{'name': 'Gizmoz', 'founded_year': 2003}
{'name': 'Digg', 'founded_year': 2004}
{'name': 'Wetpaint', 'founded_year': 2005}


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [ ]:
# Your Code
cursor = db.companies.find({"number_of_employees": {"$gt":5000}}).sort([("number_of_employees", ASCENDING)]).limit(20)

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [20]:
# Your Code
cursor = db.companies.find({"$and":[{"number_of_employees": {"$lt":1000}},{"founded_year": {"$lt":2005}}]}).sort([("number_of_employees", ASCENDING)]).limit(10)
for company in cursor:
    print(company)
    break

{'_id': ObjectId('52cdef7c4bab8bd675297d93'), 'name': 'Fox Interactive Media', 'permalink': 'fox-interactive-media', 'crunchbase_url': 'http://www.crunchbase.com/company/fox-interactive-media', 'homepage_url': 'http://www.newscorp.com', 'blog_url': '', 'blog_feed_url': '', 'twitter_username': 'twitterapi', 'category_code': 'web', 'number_of_employees': 0, 'founded_year': 1979, 'founded_month': 6, 'founded_day': 1, 'deadpooled_year': None, 'deadpooled_month': None, 'deadpooled_day': None, 'deadpooled_url': '', 'tag_list': '', 'alias_list': None, 'email_address': '', 'phone_number': '', 'description': '', 'created_at': 'Thu May 31 21:46:57 UTC 2007', 'updated_at': 'Mon Aug 19 17:13:27 UTC 2013', 'overview': '<p>Fox Interactive Media (FIM) oversees <a href="http://www.crunchbase.com/company/newscorporation" title="News Corporation">News Corporation</a>&#8217;s Internet business operations.</p>', 'image': {'available_sizes': [[[150, 71], 'assets/images/resized/0001/0824/10824v1-max-150x150

### 6. All the companies that don't include the `partners` field.

In [28]:
# Your Code
cursor = db.companies.find({"partners": {"$exists":False}})
for company in cursor:
    print(company)
    break
# No hay

### 7. All the companies that have a null type of value on the `category_code` field.

In [29]:
# Your Code
cursor = db.companies.find({"category_code": {"$type":["null"]}})
for company in cursor:
    print(company)
    break

{'_id': ObjectId('52cdef7c4bab8bd6752980f6'), 'name': 'Collective', 'permalink': 'collective', 'crunchbase_url': 'http://www.crunchbase.com/company/collective', 'homepage_url': None, 'blog_url': None, 'blog_feed_url': None, 'twitter_username': None, 'category_code': None, 'number_of_employees': None, 'founded_year': None, 'founded_month': None, 'founded_day': None, 'deadpooled_year': None, 'deadpooled_month': None, 'deadpooled_day': None, 'deadpooled_url': None, 'tag_list': None, 'alias_list': None, 'email_address': None, 'phone_number': None, 'description': None, 'created_at': 'Thu Sep 26 13:15:02 UTC 2013', 'updated_at': 'Thu Sep 26 13:15:02 UTC 2013', 'overview': None, 'image': None, 'products': [], 'relationships': [], 'competitions': [], 'providerships': [], 'total_money_raised': '$0', 'funding_rounds': [], 'investments': [], 'acquisition': None, 'acquisitions': [], 'offices': [], 'milestones': [], 'ipo': None, 'video_embeds': [], 'screenshots': [], 'external_links': [], 'partners

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [31]:
# Your Code
cursor = db.companies.find({"number_of_employees":{"$in":[i for i in range(100,1000)]}},{"name":1,"number_of_employees":1,"_id":0})
for company in cursor:
    print(company)
    break

{'name': 'AdventNet', 'number_of_employees': 600}


### 9. Order all the companies by their IPO price in a descending order.

In [ ]:
# Your Code
cursor = db.companies.sort([("number_of_employees", DESCENDING)])


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [34]:
# Your Code
cursor = db.companies.aggregate([{"$sort":{"number_of_employees":-1}}]).limit(10)

for company in cursor:
    print(company)
    break

OperationFailure: Sort exceeded memory limit of 104857600 bytes, but did not opt in to external sorting. Aborting operation. Pass allowDiskUse:true to opt in.

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [38]:
# Your Code
cursor = db.companies.find({"founded_month":{"$gt": 6}}).limit(1000)
for company in cursor:
    print(company)
    break

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'), 'name': 'Wetpaint', 'permalink': 'abc2', 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint', 'homepage_url': 'http://wetpaint-inc.com', 'blog_url': 'http://digitalquarters.net/', 'blog_feed_url': 'http://digitalquarters.net/feed/', 'twitter_username': 'BachelrWetpaint', 'category_code': 'web', 'number_of_employees': 47, 'founded_year': 2005, 'founded_month': 10, 'founded_day': 17, 'deadpooled_year': 1, 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system', 'alias_list': '', 'email_address': 'info@wetpaint.com', 'phone_number': '206.859.6300', 'description': 'Technology Platform Company', 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27), 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013', 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for digital publishers. Wet

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [41]:
# Your Code
cursor = db.companies.find({"$and":[{"founded_year": {"$lt":2000}},{"acquisition.price_amount": {"$gt":10000000}}]})
for company in cursor:
    print(company)
    break

{'_id': ObjectId('52cdef7c4bab8bd675297d90'), 'name': 'Postini', 'permalink': 'postini', 'crunchbase_url': 'http://www.crunchbase.com/company/postini', 'homepage_url': 'http://postini.com', 'blog_url': '', 'blog_feed_url': '', 'twitter_username': None, 'category_code': 'web', 'number_of_employees': None, 'founded_year': 1999, 'founded_month': 6, 'founded_day': 2, 'deadpooled_year': None, 'deadpooled_month': None, 'deadpooled_day': None, 'deadpooled_url': None, 'tag_list': '', 'alias_list': None, 'email_address': '', 'phone_number': '888.584.3150', 'description': None, 'created_at': 'Fri Jun 08 12:19:51 UTC 2007', 'updated_at': 'Sat Aug 13 18:02:34 UTC 2011', 'overview': '<p>Postini focuses on two main issues: security and compliance. Postini states that it handles more than 1 billion messages everyday and protects more than 35,000 businesses worldwide.</p>\n\n<p>Postini offers solutions that protect your company from malicious internet attacks. The Postini Communications Suite eliminat

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [42]:
# Your Code
cursor = db.companies.find({"acquisition.acquired_year": {"$gt":2010}},{"name":1,"acquisition":1,"_id":0}).sort([("acquisition.price_amount",ASCENDING)])
for company in cursor:
    print(company)
    break

{'name': 'Geni', 'acquisition': {'price_amount': None, 'price_currency_code': 'USD', 'term_code': None, 'source_url': 'http://techcrunch.com/2012/11/28/all-in-the-family-myheritage-buys-former-yammer-stablemate-geni-com-raises-25m/', 'source_description': 'MyHeritage acquires Geni and $25M to build family tree of the whole world', 'acquired_year': 2012, 'acquired_month': 11, 'acquired_day': 28, 'acquiring_company': {'name': 'MyHeritage', 'permalink': 'myheritage'}}}


### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [44]:
# Your Code
cursor = db.companies.aggregate([{"$sort":{"founded_year":-1}}],{"name":1,"founded_year":1,"_id":0}).limit(10)

for company in cursor:
    print(company)
    break

AttributeError: 'dict' object has no attribute '_txn_read_preference'

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [45]:
# Your Code
cursor = db.companies.find({"founded_day":{"$in":[i for i in range(1,8)]}}).sort([("acquisition.price_amount",DESCENDING)]).limit(10)
for company in cursor:
    print(company)
    break

{'_id': ObjectId('52cdef7d4bab8bd6752989a1'), 'name': 'Netscape', 'permalink': 'netscape', 'crunchbase_url': 'http://www.crunchbase.com/company/netscape', 'homepage_url': 'http://netscape.aol.com', 'blog_url': '', 'blog_feed_url': '', 'twitter_username': '', 'category_code': 'software', 'number_of_employees': None, 'founded_year': 1994, 'founded_month': 4, 'founded_day': 4, 'deadpooled_year': None, 'deadpooled_month': None, 'deadpooled_day': None, 'deadpooled_url': None, 'tag_list': None, 'alias_list': '', 'email_address': '', 'phone_number': '', 'description': '', 'created_at': 'Tue May 06 00:27:28 UTC 2008', 'updated_at': 'Thu Nov 14 00:57:06 UTC 2013', 'overview': '<p>Netscape Communications Corporation offers development, marketing, sale, and support of enterprise software solutions. Its products include e-commerce infrastructure and e-commerce applications targeted primarily at corporate intranets and extranets, and Internet. The company was formerly known as Mosaic Communications

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [47]:
# Your Code
cursor = db.companies.find({"$and": [{"category_code":{"$eq":"web"}},{"number_of_employees":{"$gt":4000}}]}).sort([("number_of_employees",ASCENDING)])
for company in cursor:
    print(company)
    break

{'_id': ObjectId('52cdef7c4bab8bd67529822a'), 'name': 'Expedia', 'permalink': 'expedia', 'crunchbase_url': 'http://www.crunchbase.com/company/expedia', 'homepage_url': 'http://www.expedia.com', 'blog_url': '', 'blog_feed_url': '', 'twitter_username': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400, 'founded_year': 1996, 'founded_month': None, 'founded_day': None, 'deadpooled_year': None, 'deadpooled_month': None, 'deadpooled_day': None, 'deadpooled_url': None, 'tag_list': 'travel, flights, hotels, tickets, airline, vacation, travel-comparison, travel-search', 'alias_list': '', 'email_address': '', 'phone_number': '1-800-EXPEDIA', 'description': '', 'created_at': 'Sun Feb 10 03:29:12 UTC 2008', 'updated_at': 'Sun Dec 08 07:03:33 UTC 2013', 'overview': '<p>Expedia.com is an Internet-based travel agency and a part of Expedia, Inc.. It books airline tickets, hotel reservations, car rentals, cruises, vacation packages, and various attractions and services via the World Wide W

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [49]:
# Your Code
cursor = db.companies.find({"$and": [{"acquisition.price_amount":{"$gt":10000000}},{"acquisition.price_currency_code":{"$eq":"EUR"}}]})
for company in cursor:
    print(company)
    break

{'_id': ObjectId('52cdef7c4bab8bd675297f02'), 'name': 'ZYB', 'permalink': 'zyb', 'crunchbase_url': 'http://www.crunchbase.com/company/zyb', 'homepage_url': 'http://zyb.com', 'blog_url': 'http://blog.zyb.com/', 'blog_feed_url': 'http://blog.zyb.com/index.php/feed/', 'twitter_username': None, 'category_code': 'mobile', 'number_of_employees': 25, 'founded_year': None, 'founded_month': None, 'founded_day': None, 'deadpooled_year': None, 'deadpooled_month': None, 'deadpooled_day': None, 'deadpooled_url': None, 'tag_list': 'mobile, social-networking, socialnetwork', 'alias_list': '', 'email_address': 'contact@zyb.com', 'phone_number': '', 'description': 'Mobile Social Networking', 'created_at': 'Wed Aug 22 05:14:51 UTC 2007', 'updated_at': 'Tue Jul 02 03:00:50 UTC 2013', 'overview': '<p>ZYB is a mobile phone utility and social networking site. ZYB started in 2005 as a european service to provide easy back-up services for mobile phones. Using this information, they have built a social network

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code